# OMI - Osservatorio del Mercato Immobiliare

This notebook demonstrates the OMI (Real Estate Market Observatory) database created from Agenzia delle Entrate datasets.

The data is imported using the `scripts/omi_import.py` script which:
1. Reads QI (Quotazioni Immobiliari) files containing market values and zone information
2. Reads VCN (Volumi Compravendite Nazionali) files containing national sales volumes
3. Normalizes column names and consolidates data across multiple years
4. Creates a SQLite database with proper indexes for efficient querying

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path

DB_PATH = Path('/mnt/mobile/data/AdE/OMI/omi.sqlite')

## Database Schema

The database contains 6 normalized tables:

### Quotazioni (Market Quotes)
- **quotazioni_valori**: Market values by property type, zone, and semester (2004-2024)
- **quotazioni_zone**: OMI zone definitions and characteristics

### Compravendite (Sales Transactions)
- **comuni**: Municipality reference data with market size classification
- **compravendite_residenziali**: Residential property sales by size category
- **compravendite_commerciali**: Commercial property sales (offices, shops, warehouses)
- **compravendite_pertinenze**: Pertinence sales (garages, storage)

In [ ]:
# Connect to database and show table counts
conn = sqlite3.connect(DB_PATH)

tables = pd.read_sql("""
    SELECT name as table_name 
    FROM sqlite_master 
    WHERE type='table'
    ORDER BY name
""", conn)

for table in tables['table_name']:
    count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table}", conn).iloc[0]['cnt']
    print(f"{table}: {count:,} rows")

## Quotazioni Valori (Market Values)

This table contains property market values (EUR/m2) for different property types across Italy, organized by OMI zones.

Key columns:
- `link_zona`: Unique zone identifier
- `tipologia`: Property type (e.g., "Abitazioni civili", "Box", "Uffici")
- `prezzo_min`, `prezzo_max`: Price range per m2
- `locazione_min`, `locazione_max`: Monthly rent range per m2
- `anno`, `semestre`: Time period

In [ ]:
# Sample market values for Milan
query = """
SELECT 
    comune, zona, tipologia, stato_conservazione,
    prezzo_min, prezzo_max, anno, semestre
FROM quotazioni_valori
WHERE comune = 'MILANO' 
    AND tipologia = 'Abitazioni civili'
    AND anno = 2024
    AND semestre = 1
ORDER BY prezzo_max DESC
LIMIT 10
"""
pd.read_sql(query, conn)

In [ ]:
# Average residential prices by region (latest semester)
query = """
SELECT 
    regione,
    ROUND(AVG(prezzo_min), 0) as avg_prezzo_min,
    ROUND(AVG(prezzo_max), 0) as avg_prezzo_max,
    COUNT(*) as num_zones
FROM quotazioni_valori
WHERE tipologia = 'Abitazioni civili'
    AND anno = 2024 AND semestre = 1
GROUP BY regione
ORDER BY avg_prezzo_max DESC
"""
pd.read_sql(query, conn)

## Quotazioni Zone (OMI Zones)

This table defines the OMI zones used for property valuation. Each zone has:
- A unique `link_zona` identifier
- A description (`zona_descrizione`)
- A fascia (band: B=central, C=semi-central, D=peripheral, E=suburban, R=rural)

In [ ]:
# Zone distribution by fascia for Rome
query = """
SELECT 
    fascia,
    COUNT(DISTINCT link_zona) as num_zones,
    GROUP_CONCAT(DISTINCT zona, ', ') as zones
FROM quotazioni_zone
WHERE comune = 'ROMA'
    AND anno = 2024 AND semestre = 1
GROUP BY fascia
ORDER BY fascia
"""
pd.read_sql(query, conn)

## Compravendite (Sales Transactions)

These tables contain NTN (Numero di Transazioni Normalizzate) - normalized transaction counts by municipality and year.

In [ ]:
# Top 10 municipalities by residential sales in 2024
query = """
SELECT 
    c.regione, c.provincia, c.comune,
    r.ntn_totale as totale_vendite,
    r.ntn_fino_50mq as fino_50mq,
    r.ntn_50_85mq as mq_50_85,
    r.ntn_85_115mq as mq_85_115,
    r.ntn_115_145mq as mq_115_145,
    r.ntn_oltre_145mq as oltre_145mq
FROM compravendite_residenziali r
JOIN comuni c ON r.cod_comune = c.cod_comune AND r.anno = c.anno
WHERE r.anno = 2024
ORDER BY CAST(REPLACE(r.ntn_totale, ',', '.') AS REAL) DESC
LIMIT 10
"""
pd.read_sql(query, conn)

In [ ]:
# Residential sales trend by year (national total)
query = """
SELECT 
    anno,
    ROUND(SUM(CAST(REPLACE(ntn_totale, ',', '.') AS REAL)), 0) as totale_ntn
FROM compravendite_residenziali
WHERE ntn_totale IS NOT NULL AND ntn_totale != ''
GROUP BY anno
ORDER BY anno
"""
trend = pd.read_sql(query, conn)
trend

In [ ]:
# Commercial property sales by category (2024)
query = """
SELECT 
    regione,
    ROUND(SUM(CAST(REPLACE(ntn_uffici, ',', '.') AS REAL)), 0) as uffici,
    ROUND(SUM(CAST(REPLACE(ntn_negozi_lab, ',', '.') AS REAL)), 0) as negozi,
    ROUND(SUM(CAST(REPLACE(ntn_produttivo, ',', '.') AS REAL)), 0) as produttivo
FROM compravendite_commerciali
WHERE anno = 2024
GROUP BY regione
ORDER BY uffici DESC
LIMIT 10
"""
pd.read_sql(query, conn)

## Importing/Updating Data

To import or update the OMI data, run the import script:

```bash
python scripts/omi_import.py --data-dir /path/to/omi/zips --output /path/to/omi.sqlite
```

The script expects QI and VCN ZIP files from Agenzia delle Entrate in the data directory.

In [ ]:
conn.close()